In [26]:
import pandas as pd

# Load the cleaned data
df = pd.read_csv('T:\Data Collection Teamlink Project\pythonProject\cleaned_fake.csv')

# Data Cleaning (if necessary)



In [27]:
df.columns

Index(['Property ID', 'Name', 'Company', 'Email Address', 'Phone Number',
       'Zip Code', 'Address', 'City', 'State', 'Suburb', 'Latitude',
       'Longitude', 'Did you find what you were looking for',
       'Overall Condition of the Property', 'Interaction with the agent',
       'Satisfaction with the inspection process', 'Likelihood to Buy or Rent',
       'What did you like most?', 'How well did the property meet your needs?',
       'View Similar Properties', 'Additional Comments'],
      dtype='object')

In [28]:
df.rename(columns={
    'Overall Condition of the Property': 'Property_Condition',
    'Interaction with the agent': 'Agent_Interaction',
    'Satisfaction with the inspection process': 'Inspection_Satisfaction',
    'Likelihood to Buy or Rent': 'Buy_Rent_Likelihood',
    'What did you like most?': 'Liked_Features',
    'Additional Comments': 'Comments'
}, inplace=True)

In [29]:
# Fill missing values with the median or mode, or drop rows with missing values
df['Buy_Rent_Likelihood'].fillna(df['Buy_Rent_Likelihood'].median(), inplace=True)
df['Property_Condition'].fillna(df['Property_Condition'].mode()[0], inplace=True)
df['Inspection_Satisfaction'].fillna(df['Inspection_Satisfaction'].mode()[0], inplace=True)

# Alternatively, drop rows with missing values in these columns
# df.dropna(subset=['Buy_Rent_Likelihood', 'Property_Condition', 'Inspection_Satisfaction'], inplace=True)


C:\Users\shiva\AppData\Local\Temp\ipykernel_14040\3826349141.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Buy_Rent_Likelihood'].fillna(df['Buy_Rent_Likelihood'].median(), inplace=True)
C:\Users\shiva\AppData\Local\Temp\ipykernel_14040\3826349141.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values 

In [30]:
import pandas as pd


# Rename columns for easier reference


# Data Cleaning (if necessary)
df.drop_duplicates(inplace=True)  # Remove duplicates
df.fillna({
    'Property_Condition': df['Property_Condition'].mode()[0],
    'Agent_Interaction': df['Agent_Interaction'].median(),
    'Inspection_Satisfaction': df['Inspection_Satisfaction'].mode()[0],
    'Buy_Rent_Likelihood': df['Buy_Rent_Likelihood'].median(),
    'Liked_Features': 'Unknown',
    'Comments': 'No comments'
}, inplace=True)

# Convert categorical ratings to numerical for analysis
rating_map = {'Very Satisfied': 5, 'Satisfied': 4, 'Neutral': 3, 'Unsatisfied': 2, 'Very Unsatisfied': 1}
df['Inspection_Satisfaction'] = df['Inspection_Satisfaction'].map(rating_map)

# Property Engagement Index (PEI)
df['PEI'] = (0.4 * df['Buy_Rent_Likelihood'] +
             0.3 * df['Inspection_Satisfaction'] +
             0.3 * df['Agent_Interaction'])

# Feature Impact Score (FIS)
features = ['Bedrooms', 'Bathrooms', 'Kitchen', 'Living Space', 'Garage', 'Balcony', 'Neighborhood', 'Price']
for feature in features:
    df[feature] = df['Liked_Features'].apply(lambda x: feature in x)

feature_impact_scores = {}
for feature in features:
    feature_impact_scores[feature] = df[df[feature] == True]['Buy_Rent_Likelihood'].mean()

print("Feature Impact Scores:")
for feature, score in feature_impact_scores.items():
    print(f"{feature}: {score:.2f}")

# Agent Performance Index (API)
df['Converted'] = df['Buy_Rent_Likelihood'] > 3
conversion_rate = df['Converted'].mean()
df['API'] = (0.5 * df['Agent_Interaction'] +
             0.3 * df['Property_Condition'] +
             0.2 * conversion_rate)

# Suburb Performance Index (SPI)
suburb_grouped = df.groupby('Suburb').agg({
    'Buy_Rent_Likelihood': 'mean',
    'Property_Condition': 'mean',
    'Inspection_Satisfaction': 'mean'
})

suburb_grouped['SPI'] = (suburb_grouped['Buy_Rent_Likelihood'] *
                         suburb_grouped['Property_Condition'] *
                         suburb_grouped['Inspection_Satisfaction'])

print(suburb_grouped[['SPI']].sort_values(by='SPI', ascending=False))

# Export the KPIs to CSV files
df.to_csv('kpis_with_correct_columns.csv', index=False)
suburb_grouped.to_csv('suburb_spi_correct_columns.csv')

print("KPIs and cleaned data with correct columns saved successfully.")


Feature Impact Scores:
Bedrooms: 2.85
Bathrooms: 2.59
Kitchen: 2.75
Living Space: 3.01
Garage: 2.84
Balcony: 3.23
Neighborhood: 2.87
Price: 2.93
             SPI
Suburb          
Bondi        NaN
Chatswood    NaN
Cronulla     NaN
Liverpool    NaN
Manly        NaN
Mosman       NaN
Newtown      NaN
Parramatta   NaN
Penrith      NaN
Surry Hills  NaN
KPIs and cleaned data with correct columns saved successfully.
